# 34、AdaBoost：使用AdaBoost提升分类器的性能

在数据挖掘中，分类算法可以说是核心的算法，其中AdaBoost算法与随机森林算法一样属于分类算法中的集成算法。

集成的涵义就是集思广益，博取众长，当我们做决定的时候，先听取多个专家的意见，再做决定。集成算法通常有两种方式，分别是投票选举（bagging）和再学习（boosting）。投票选举的场景类似于把专家召集到一个会议桌前，当做一个决定的时候，让K个专家（K个模型）分别进行分类，然后选择出现最多的那个类作为最终的分类结果。再学习相当于把K个专家（K个分类器）进行加权融合，形成一个新的超级专家（强分类器），让这个超级专家做判断。

所以，我们能看出来，投票选举和再学习还是有区别的。Boosting的含义是提升，它的作用是每一次训练的时候都对上一次的训练进行改进提升，在训练的过程中K个“专家”之间是有依赖性的，当引入第K个“专家”（第K个分类器）的时候，实际上是对前K-1个专家的优化。而bagging在做投票选举的时候可以并行计算，也就是K个“专家”在做判断的时候是相互独立的，不存在依赖性。


## AdaBoost的工作原理

了解了集成算法的两种模式之后，再来理解AdaBoost算法。

AdaBoost的英文全称是Adaptive Boosting，中文含义是自适应提升算法。它由Freund等人遇1995年提出，是对Boosting算法的一种实现。

什么是Boosting算法呢？Boosting算法是集成算法中的一种，同时也是一类算法的总称。这类算法通过训练多个弱分类器，将它们组合成一个强的分类器，也就是我们俗话说的“三个臭皮匠，顶个诸葛亮”。为什么要这样做呢？因为臭皮匠好训练，而诸葛亮却是不好求。因此要打造一个诸葛亮，最好的方式就是训练多个臭皮匠，然后让这些臭皮匠组合起来，这样往往会得到很好的效果。这就是Boosting算法的原理。
![](AdaBoost原理1.jpg)

我们可以用上面的图表示最终的得到的强分类器，你能看出它是通过一系列的弱分类器根据不同的权重组合而成的

假设弱分类器Gi(x),它在强分类器中的权值αi,那么就可以得出强分类器f(x):
![](AdaBoost公式1.png)

有了这个公式，为了求解强分类器，我们需要关注两个问题：

- 1、如何得到弱分类器，也就是在每次的迭代的训练过程中，如何得到最优的若分类器？
- 2、每个弱分类器在强分类器中的权重是如何计算的？

我们先来看一下第二个问题。实际上在一个由K个弱分类器组成的强分类器中，如果弱分类器的分类效果好，那么权重应该比较大，如果分类器的分类效果一般，权重应该降低。所以我们需要基于这个若分类器对样本的分类错误率来决定他的权重，用公式表示就是：
![](AdaBoost公式2.png)
其中ei代表的是第i个分类器的分类错误率。

然后我们再来看一下第一个问题，如何在每次训练迭代的过程中选择最优的弱分类器？

实际上，AdaBoost算法是通过改变样本的数据分布来实现的。AdaBoost会判断每次训练的样本是否正确分类，对于正确分类的样本，降低它的权重，对于被错误分类的样本，增加它的权重。再基于上一次得到的分类准确率，来确定这次训练样本中每个样本的权重。然后将修改过的权重的新数据集传递给下一层的分类器进行训练。这样做的好处是，通过每一轮的训练样本的动态权重，可以让训练的焦点集中到难分类的样本上，最终得到的弱分类器的组合更加容易得到更高的分类准确率。

我们可以使用Dk+1代表第k+1轮训练中，样本的权重集合，其中Wk+1,1代表第k+1轮中第一个样本的权重，以此类推Wk+1,N代表第k+1轮中第N个样本的权重，因此用公式表示为：
![](AdaBoost公式3.png)

第k+1轮中的样本权重，是根据该样本在第k轮的权重以及第k个分类器的准确率而定，具体的公式为：
![](AdaBoost公式4.png)

**注：** 这里Zk是归一化因子，使得Dk+1成为一个概率分布。yi属于标记集合{-1,+1}，1代表分类正确，-1代表分类错误。

## AdaBoost 算法示例

了解AdaBoost的工作原理之后，我们可以看一个例子，假设我们有10个训练样本，如下所示：

|X|0|1|2|3|4|5|6|7|8|9|
|--|--|--|--|--|--|--|--|--|--|--|
|Y|1|1|1|-1|-1|-1|1|1|1|-1|

现在希望通过AdaBoost构建一个强分类器。

该如何做？可以按照上面的AdaBoost工作原理进行模拟：

首先在第一轮中，我们得到10个样本的权重1/10,即开始的10个样本权重一致，D1=(0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1).

假设我们有单个分类器：
![](AdaBoost公式5.png)

我们可以知道分类器f1的错误率为0.3，也就是x取值6,7,8时分类器错误；分类器f2的错误率为0.4，即x取值0、1、2、9时分类错误；分类器f3的错误率为0.3，即x取值为3,4,5时分类错误。

这3个分类器中，f1、f3分类器的错曲率最低，因此我们选择f1或者f3作为最优分类器，假设我们选择f1分类器作为最优分类器，即第一轮训练得到：
![](AdaBoost公式6.png)

根据分类器权重公式得到：
![](AdaBoost公式7.png)

然后我们对下一轮的样本更新求权重值，代入Wk+1,i和Dk+1的公式，可以得到新的权重矩阵：D2=(0.0715,0.0715,0.0715,0.0715,0.0715,0.0715,0.1666,0.1666,0.1666,0.0715).

第二轮训练中，我们继续统计三个分类器的准确率，可以得到分类器f1的错误率为0.1666*3，也就是x取值为6、7、8时分类错误。分类器f2的错误率为0.0715*4，即x取值为0,1,2,9时分类错误。分类器f3的错误率为0.0715*3，即x取值为3，4，5时候分来错误。

在这3个分类器中，f3分类器的错误率最低，因此我们选择f3作为第二轮训练的最优分类器，即：
![](AdaBoost公式8.png)
根据分类器权重公式得到：
![](AdaBoost公式9.png)

同样，我们对下一轮的样本更新求权重值，代入Wk+1,i和Dk+1的公式，可以得到D3=(0.0455,0.0455,0.0455,0.1667,0.1667,0.1667,0.1060,0.1060,0.1060,0.0455).

在第三轮的训练中，我们继续统计三个分类器的准确率，可以得到分类器f1的错误率为0.1060*3，也就是x取值6,7,8时分类错误。分类器f2的错误率为0.0455*4，即x取值为0、1、2、9时分类错误。分类器f3的错误率为0.1667*3，即x取值为3,4,5时分类错误。

在这三个分类器中，f2分类器的错误率最低，因此我们选择f2作为第三轮训练的最优分类器，即：
![](AdaBoost公式10.png)

我们根据分类器权重公式得到：
![](AdaBoost公式11.png)

假设我们只进行3轮训练，选择3个弱分类器，组成一个强的分类器，那么最终的强分类器G(x)=0.4236G1(x) + 0.6496G2(x) + 0.7514G3(x).

实际上AdaBoost算法是一个框架，你可以指定任意的分类器，通常我们可以采用CART分类器作为弱分类器。通过上面的这个示例运算，我们可以体会到AdaBoost的计算流程。

## 总结
1、对于AdaBoost算法的原理，我们可以理解成为一种集成算法，通过训练不同的弱分类器，将这些弱分类器集成起来形成一个强的分类器。在每一轮的训练中手绘加入一个新的弱分类器，直到达到足够低的错误率或者达到指定的最大迭代次数为止。实际上每一次迭代都会引入一个新的弱分类器（这个分类器是每一次迭代中计算出来的，是新的分类器，不是事先准备好的）。

2、在弱分类器的集合中，我们不用担心分类器太弱了。实际上它只是需要比随机猜想的效果略好一些即可。如果随机猜想的准确率是50%的话，那么每个弱分类器的准确率只要大于50%就可以使用。AdaBoost的强大在于迭代训练的机制，这样通过K个“臭皮匠”的组合也可以得到一个“诸葛亮”（强分类器）。

3、当然在每一轮的训练中，我们需要从众多的“臭皮匠”中选择一个拔尖的，也就是这一轮训练评比中的最优“臭皮匠”，对应的就是错误率最低的分类器。当然每一轮的样本的权重都会发生变化，这样做的目的是为了让之前错误分类的样本得到更多概率的重复训练的机会。同样的原理在我们的学习生活中也是经常出现的，比如善于利用错题本来提升学习效率和学习成绩。

4、AdaBoost算法
- 集成算法
    - Boosting算法：每一次训练的时候都会对上一次的训练进行改进提升
    - Bagging算法：投票选举机制，让K个专家分别进行分类，然后选择出现次数最多的那个类作为最终的分类结果
- 工作原理
    - 强分类器：一系列弱分类器根据不同的权重组合而成。
    - 弱分类器在强分类器中的权重计算。
    - 每一轮样本的权重更新。
    - 算法示例：10个训练样本（X,Y），3个基础的分类器，如何构造一个强分类器。
    
![](AdaBoost总结.png)

### AdaBoost算法流程

- 1、一开始，给训练数据中的每一个样本，赋予其一个权重，权重都初始化为相等的值。如（1/样本数量）
- 2、首先在训练数据中训练处一个弱分类器并计算该分类器的错误率，选取错误率小的分类器，并基于分类器错误率计算其权重值alpha
- 3、在分类器的第二次训练中，将会重新调整每个样本的权重，其中第一次分对的样本的权重将会降低，而第一次分错的样本的权重将会提高。然后在同一数据集上再次训练弱分类器。得出第二个错误率小的分类器，并基于错误率计算权值。
- 4、重复“重新分配样本权重--计算分类器错误率--选取分类器--计算分类器权重”
- 5、最后将每次训练得到的分类器融合起来，作为最后的决策分类器。

### 弱分类器和强分类器
- 弱分类器，是指基础分类器，正确率略高于50%的那种。
- 强分类器是通过训练出多个弱分类器，并赋值权重，最后形成弱分类器+权重的模型。